![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and custom foundation model to make simple chat conversation

#### Disclaimers

- Chat for custom foundation models is supported since CPD 5.1.2.
- Use only Projects and Spaces that are available in watsonx context.
- To enable chat support, the custom model content (tokenizer_config.json) must include a chat template.
- Chat support is limited to the models that use software spec: `watsonx-cfm-caikit-1.1`.

## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase chat support for custom foundation models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to chat with custom foundation models.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to watsonx.ai Runtime

Authenticate the watsonx.ai Runtime Service on IBM Cloud Pak for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate watsonx.ai Runtime services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have project already created follow bellow steps.

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
project_id = 'PASTE YOUR PROJECT ID HERE'

Create an instance of APIClient with authentication details.

In [5]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id)

<a id="models"></a>
## Set up the Custom Foundation Model on `watsonx.ai`


### Store the model

Specify software specification. Note that only `watsonx-cfm-caikit-1.1` is valid for Chat with BYOM.

In [6]:
sw_spec_id = client.software_specifications.get_id_by_name('watsonx-cfm-caikit-1.1')

Specify the `model_id` of the model you will use for the chat with tools. The model has to be available on your CPD instance.

In [7]:
model_id = 'deepseek-r1-distill-llama-8b'

There is also a possibility to list available custom models with their specifications.

In [8]:
from ibm_watsonx_ai.foundation_models_manager import FoundationModelsManager

fm_manager = FoundationModelsManager(client)

fm_manager.get_custom_model_specs()

Publish the model.

In [9]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'Custom Deepseek FM',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
    client.repository.ModelMetaNames.TYPE: client.repository.ModelAssetTypes.CUSTOM_FOUNDATION_MODEL_1_0
}

model_details = client.repository.store_model(
    model=model_id,
    meta_props=metadata,
)

### Deploy the model

Specify hardware specification.

In [10]:
hw_spec_id = client.hardware_specifications.get_id_by_name("WX-S")

Get `stored_model_id` from `model_detais` and specify `deployment_metadata`.

In [ ]:
stored_model_id = client.repository.get_model_id(model_details)

deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of custom model",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"id": hw_spec_id},
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.SERVING_NAME : 'custom_fm',
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Deploy of custom deepseek model"
}

Create deployment.

In [ ]:
deployment_details = client.deployments.create(
    artifact_id=stored_model_id,
    meta_props=deployment_metadata,
)

Get `deployment_id` from `deployment_details`.

In [ ]:
deployment_id = client.deployments.get_id(deployment_details)

### Define the model parameters

You might need to adjust model `parameters` depending on the model you use.

In [14]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-------------------+----------------------------------------+------------------------------+
| PARAMETER         | TYPE                                   | EXAMPLE VALUE                |
+===================+========================================+==============================+
| frequency_penalty | float, NoneType                        | 0.5                          |
+-------------------+----------------------------------------+------------------------------+
| logprobs          | bool, NoneType                         | True                         |
+-------------------+----------------------------------------+------------------------------+
| top_logprobs      | int, NoneType                          | 3                            |
+-------------------+----------------------------------------+------------------------------+
| presence_penalty  | float, NoneType                        | 0.3                          |
+-------------------+---------------------------------------

In [15]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [16]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    deployment_id=deployment_id,
    api_client=client,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

#### Work with a simple chat message

In [17]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(messages=messages)

In [ ]:
print(simple_chat_response["choices"][0]["message"]["content"])

I need to find the sum of 1 and 1.

First, I recognize that adding two numbers together involves combining their values.

Since both numbers being added are 1, I can simply count two objects, each representing one of the numbers.

By counting them, I determine that the total is 2.

Therefore, 1 plus 1 equals 2.
</think>

Sure! Let's solve the problem step by step.

**Problem:** What is \(1 + 1\)?

**Solution:**

To find the sum of 1 and 1, follow these simple steps:

1. **Start with the first number:** 1.
   
2. **Add the second number to it:** \(1 + 1\).

3. **Combine them:** When you add 1 and 1 together, you're essentially counting two items or quantities.

4. **Calculate the result:** \(1 + 1 = 2\).

Therefore, the final answer is \(\boxed{2}\).


#### Work with a simple chat message using chat_stream

In [19]:
messages = [
    {
        "role": "user", 
        "content": "What IBM mainly does?"
    }
]

simple_chat_stream_response = model.chat_stream(messages=messages)

In [20]:
for chunk in simple_chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Okay, so I'm trying to figure out what IBM mainly does. I've heard the name IBM before—it's one of those big tech companies, right? But I'm not exactly sure about the specifics. Let me think about what I know and try to piece it together.

First off, IBM stands for International Business Machines. The name sounds old-fashioned, like from the 50s or 60s. I think they were big back in the days of mainframe computers. But now, in the age of personal computers and the internet, what do they do? I remember hearing that IBM made the first personal computer, the PC, which was a big deal in the 1980s. That must have been their entry into the consumer market.

So, they make computers and related technology. But is that all? I believe they also deal with software. I think IBM has developed a lot of software, maybe even some operating systems. Wait, wasn't there something called DOS? I think IBM had a version of DOS which was used a lot in the early days of PCs. So, they not only sell hardware bu

#### Work with an advanced chat message

In [21]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(messages=messages)

In [22]:
print(advanced_chat_response["choices"][0]["message"]["content"])

Alright, so the user just asked where the 2020 World Series was played. Let me break this down.

First, in 2020, there was some uncertainty because of the ongoing COVID-19 pandemic. Major League Baseball had to make some tough decisions to keep the season going. They decided to have all the games played at one venue to minimize travel and reduce exposure risks.

Now, the home field for the World Series is usually determined by the arrangement of the teams. Since the World Series is between two teams, often the series is alternately hosted by each team. But in 2020, they broke this tradition. They chose the Los Angeles Dodgers' home stadium, Dodger Stadium, as the only venue for all the games.

So, the user might not be aware that the World Series wasn't held at different venues due to the pandemic. They might think it's standard to move around, but in this special case, it was all in LA.

I should make sure to mention why it was held in LA specifically, highlighting the pandemic safety

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to chat with custom foundation models and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Karol Zmorski**, Software Engineer at watsonx.ai.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.